In [47]:
from logicqubit.logic import *
from cmath import *
import numpy as np
import sympy as sp
import scipy
from random import randrange
from scipy.optimize import *
import matplotlib.pyplot as plt

In [48]:
gates = Gates(1)

ID = gates.ID()
X = gates.X()
Y = gates.Y()
Z = gates.Z()

In [73]:
III = ID.kron(ID).kron(ID)
XXX = X.kron(X).kron(X)
YYY = Y.kron(Y).kron(Y)
ZZZ = Z.kron(Z).kron(Z)
IZZ = ID.kron(Z).kron(Z)
ZZI = ID.kron(Z).kron(ID)
sig_izz = [IZZ.get()[i,i] for i in range(len(IZZ.get()))]
sig_zzi = [ZZI.get()[i,i] for i in range(len(ZZI.get()))]
sig_izz

[1, -1, -1, 1, 1, -1, -1, 1]

In [128]:
H = III*2 + YYY*4 + IZZ*3 + ZZZ*2

min(scipy.linalg.eig(H.get())[0])

(-5.47213595499958+0j)

In [129]:
def _ansatz(reg, params):
    n_qubits = len(reg)
    depth = n_qubits
    for i in range(depth):
        reg[1].CNOT(reg[0])
        for j in range(n_qubits):
            reg[i].RY(params[j])
            
def ansatz(reg, params):
    n_qubits = len(reg)
    depth = n_qubits
    for i in range(depth):
        for j in range(n_qubits):
            if(j < n_qubits-1):
                reg[j+1].CNOT(reg[j])
            reg[i].RY(params[j])
            
def ansatz_3q(q1, q2, q3, params):
    q1.RY(params[0])
    q2.RY(params[1])
    q3.RY(params[2])
    q2.CNOT(q1)
    q3.CNOT(q2)
    q1.RX(params[3])
    q2.RX(params[4])
    q3.RX(params[5])
    q2.CNOT(q1)
    q3.CNOT(q2)
    q1.RY(params[6])
    q2.RY(params[7])
    q3.RY(params[8])
    q2.CNOT(q1)
    q3.CNOT(q2)

In [130]:
def expectation_3q(params):
    logicQuBit  = LogicQuBit(3)
    q1 = Qubit()
    q2 = Qubit()
    q3 = Qubit()

    ansatz_3q(q1,q2,q3,params)
    #ansatz([q1,q2,q3],params)
    psi = logicQuBit.getPsi()
    
    return (psi.adjoint()*H*psi).get()[0][0]

minimum = minimize(expectation_3q, [0,0,0,0,0,0,0,0,0], method='Nelder-Mead', options={'xtol': 1e-10, 'ftol': 1e-10})
print(minimum)

 final_simplex: (array([[ 0.12921139, -1.57086317,  1.57080437, -1.12420308, -3.03704134,
         1.45367023, -0.29225308,  1.95750013,  1.5707652 ],
       [ 0.12924501, -1.57078486,  1.57088293, -1.1242658 , -3.0369472 ,
         1.4536376 , -0.29225486,  1.95729195,  1.57079917],
       [ 0.12916145, -1.5708624 ,  1.57083772, -1.12424803, -3.03717848,
         1.45370456, -0.29226349,  1.95760535,  1.57082513],
       [ 0.12919184, -1.57076394,  1.57080683, -1.12432742, -3.03703452,
         1.45363379, -0.29218685,  1.95743054,  1.57083746],
       [ 0.12922316, -1.5708506 ,  1.57084737, -1.12431734, -3.03698036,
         1.45369964, -0.29223934,  1.95744884,  1.57076656],
       [ 0.1291532 , -1.57080607,  1.57081857, -1.12425794, -3.03700681,
         1.45368743, -0.29227974,  1.95751193,  1.57072828],
       [ 0.12924718, -1.57072935,  1.5707414 , -1.12422313, -3.03688703,
         1.45354606, -0.29216359,  1.95727596,  1.57078718],
       [ 0.12910942, -1.57073622,  1.57084631

In [131]:
#ZZZ
#000 =  1
#001 = -1
#010 = -1
#011 =  1
#100 = -1
#101 =  1
#110 =  1
#111 = -1

def expectation_value(measurements, base = np.array([1,-1,-1,1,-1,1,1,-1])):
    probabilities = np.array(measurements)
    expectation = np.sum(base * probabilities)
    return expectation

def sigma_xxx(params):
    logicQuBit  = LogicQuBit(3, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    q3 = Qubit()
    
    ansatz_3q(q1,q2,q3,params)
    
    # medidas em XX
    q1.RY(-pi/2)
    q2.RY(-pi/2)
    q3.RY(-pi/2)
    
    result = logicQuBit.Measure([q1,q2,q3])
    result = expectation_value(result)
    return result

def sigma_yyy(params):
    logicQuBit  = LogicQuBit(3, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    q3 = Qubit()
    
    ansatz_3q(q1,q2,q3,params)
    
    # medidas em YY
    q1.RX(pi/2)
    q2.RX(pi/2)
    q3.RX(pi/2)
    
    result = logicQuBit.Measure([q1,q2,q3])
    result = expectation_value(result)
    return result

def sigma_zzz(params):
    logicQuBit  = LogicQuBit(3, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    q3 = Qubit()
    
    ansatz_3q(q1,q2,q3,params)
          
    result = logicQuBit.Measure([q1,q2,q3])
    zzz = expectation_value(result)
    izz = expectation_value(result, sig_izz) # [zzz, izz] = 0
    return zzz, izz 

def expectation_energy(params):
    xxx =  sigma_xxx(params)
    yyy =  sigma_yyy(params)
    zzz, izz =  sigma_zzz(params)

    result = 2 + 4*yyy + 3*izz + 2*zzz
    return result

In [132]:
#initial_values = [random.random() for _ in range(3)]
minimum = minimize(expectation_energy, [0,0,0,0,0,0,0,0,0], method='Nelder-Mead', options={'xtol': 1e-10, 'ftol': 1e-10})
print(minimum)

 final_simplex: (array([[-1.02400892,  1.28055431,  0.05946761,  1.33300369,  0.08342998,
         2.19934652, -6.29492554,  1.83668327,  0.42674918],
       [-1.0045284 ,  1.23991259,  0.10521914,  1.37250857,  0.10442777,
         2.12314711, -6.24443198,  1.80795255,  0.41251561],
       [-0.96316889,  1.28037413, -0.02258307,  1.37967455,  0.087312  ,
         2.19233154, -6.29865919,  1.84036414,  0.42136336],
       [-1.01088839,  1.26968456,  0.02772341,  1.47440312,  0.11925323,
         2.16213358, -6.28100343,  1.81227668,  0.37292877],
       [-0.93486007,  1.27946439,  0.01883379,  1.40376204,  0.10724767,
         2.13684427, -6.31317357,  1.84579082,  0.39146723],
       [-1.01355085,  1.25750614,  0.099545  ,  1.45014256,  0.12627078,
         2.12002938, -6.27548029,  1.81035501,  0.36927212],
       [-1.07176354,  1.24202417,  0.03454186,  1.48001232,  0.11766726,
         2.1641785 , -6.1354307 ,  1.75219463,  0.36242693],
       [-0.96319626,  1.2278143 ,  0.03577467

In [133]:
def gradient(params, evaluate):
    n_params = params.shape[0]
    shift = pi/2
    gradients = np.zeros(n_params)
    
    for i in range(n_params):
        #parameter shift rule
        shift_vect = np.array([shift if j==i else 0 for j in range(n_params)])
        shift_right = params + shift_vect
        shift_left = params - shift_vect
        
        expectation_right = evaluate(shift_right)
        expectation_left = evaluate(shift_left)

        gradients[i] = expectation_right - expectation_left

    return gradients

In [134]:
params = np.random.uniform(-np.pi, np.pi, 9)
last_params = np.zeros(9)

In [135]:
lr = 0.1
err = 1
while err > 1e-3:
    grad = gradient(params, expectation_energy)
    params = params - lr*grad
    err = abs(sum(params - last_params))
    last_params = np.array(params)
    print(err) 

/tmp/ipykernel_12923/1088512639.py:15: ComplexWarning: Casting complex values to real discards the imaginary part
  gradients[i] = expectation_right - expectation_left


2.269557430510688
0.43148743285348523
0.3173515038192558
0.1843803492700199
0.04327682033010166
0.0205755822034282
0.02982539400493711
0.0242148016084765
0.01835772396267603
0.013352718021686673
0.009994564789378567
0.007462727872315965
0.005675856339475829
0.004299655543890113
0.00327869541767134
0.002487918781161013
0.0018904697303590463
0.0014302939341544803
0.001081493614157969
0.00081494892435412


In [136]:
expectation_energy(params)

(-5.472130682938133+0j)